In [2]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numba
from numba import njit
import numpy as np
from pocket_coffea.lib.parton_provenance import reverse_index_array
from pocket_coffea.lib.deltaR_matching import object_matching


# use the last b quark for the matching

In [3]:
j=0
filename ="/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/hh4b_12.root"
# filename ="/pnfs/psi.ch/cms/trivcat/store/mc/Run3Summer22EENanoAODv12/VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/508f1313-c598-48b5-984e-86cd0f02a820.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_start=j, entry_stop=j+10).events()
print("Events read:", len(events))

Events read: 10


In [4]:
# add column with the index
events.GenPart = ak.with_field(
    events.GenPart, ak.local_index(events.GenPart, axis=1), "index"
)
print(events.GenPart.index[0])
print(events.GenPart.index[1])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]

print(bquarks_last.pdgId, bquarks_last.genPartIdxMother, bquarks_last.childrenIdxG, bquarks_last.hasFlags(["isLastCopy"]))
bquark_last_children=bquarks_last.children
print(bquark_last_children.pdgId, bquark_last_children.genPartIdxMother, bquark_last_children.hasFlags(["isLastCopy"]))

b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    mask_mother = (abs(b_mother.pdgId) == 5) | ((b_mother.pdgId) == 25)

    bquarks_last = bquarks_last[mask_mother]
    b_quarks = b_quarks[mask_mother]
    b_mother = b_mother[mask_mother]
    print(
        "old: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
        "index",
        b_quarks.index,
        "fromHardProcess",
        b_quarks.hasFlags(["fromHardProcess"]),
    )
    print(
        "mother: ",
        "pdg",
        b_mother.pdgId,
        "mother_idx",
        b_mother.genPartIdxMother,
        "pt",
        b_mother.pt,
    )
    # break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    print(
        "new: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
    )

    print(ak.any(abs(b_mother.pdgId) != 25, axis=1))
    if ak.all(abs(b_mother.pdgId) == 25):
        break

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))
print(b_quarks.index, len(b_quarks.index))
print(bquarks_last.pdgId, bquarks_last.index,  bquarks_last.pt, len(bquarks_last.index), bquarks_last.hasFlags(["fromHardProcess"]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
[[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [-5, 5, 5, -5]] [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [16, 20, 18, 19], [28, 27, 25, 26]] [[[38, 39, 59, 60], [], [], []], [[], [], [], ... [[], [], [], []], [[], [], [], []]] [[True, True, True, True], [True, True, ... True, True], [True, True, True, True]]
[[[-513, 523, 11, -11], [], [], []], [[], [], ... [], [], [], []], [[], [], [], []]] [[[33, 33, 33, 33], [], [], []], [[], [], [], ... [[], [], [], []], [[], [], [], []]] [[[True, True, True, True], [], [], []], [[], ... [], [], []], [[], [], [], []]]

loop
old:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [-5, 5, 5, -5]] mother_idx [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [16, 20, 18, 19], [28, 27, 25, 26]] pt [[116, 124, 186, 13.8], [87.2, 35.4, 103, ... 75.

In [32]:
print(bquarks_last.index, len(bquarks_last.index))


[[33, 37, 40, 42], [21, 27, 32, 33], [21, ... [23, 24, 27, 29], [29, 31, 33, 34]] 10


# NUMBA

In [33]:
genpart =events.GenPart

# get last bquark copy
isB = abs(genpart.pdgId) == 5
isLast = genpart.hasFlags(["isLastCopy"])
isFirst = genpart.hasFlags(["isFirstCopy"])
isHard = genpart.hasFlags(["fromHardProcess"])

isHiggs = genpart.pdgId == 25
higgs = genpart[isHiggs & isLast & isHard]
higgs = higgs[ak.num(higgs.childrenIdxG, axis=2) == 2]

higgs = higgs[ak.argsort(higgs.pt, ascending=False)]


In [34]:
# bquarks = genpart[isB & isHard & isFirst]
bquarks = genpart[isB & isHard]
mother_bquarks = genpart[bquarks.genPartIdxMother]
print(bquarks.index, mother_bquarks.index,mother_bquarks.childrenIdxG, mother_bquarks.pdgId)
print(mother_bquarks.pdgId == 25)
bquarks_from_higgs = bquarks[mother_bquarks.pdgId == 25]

print(bquarks_from_higgs.index, bquarks_from_higgs.genPartIdxMother)
print("bquarks_from_higgs", bquarks_from_higgs.index, bquarks_from_higgs.pdgId)

[[17, 18, 19, 20, 33, 37, 40, 42], [16, ... 29], [25, 26, 27, 28, 29, 31, 33, 34]] [[15, 15, 16, 16, 17, 18, 19, 20], [14, ... 19], [23, 23, 24, 24, 28, 27, 25, 26]] [[[17, 18], [17, 18], [19, 20], [19, ... 639], [640, 641], [642], [643, 644, 645]]] [[25, 25, 25, 25, 5, -5, 5, -5], [25, 25, ... 5, -5], [25, 25, 25, 25, -5, 5, 5, -5]]
[[True, True, True, True, False, False, ... True, True, False, False, False, False]]
[[17, 18, 19, 20], [16, 17, 18, 19], [17, ... [16, 17, 18, 19], [25, 26, 27, 28]] [[15, 15, 16, 16], [14, 14, 15, 15], [15, ... [14, 14, 15, 15], [23, 23, 24, 24]]
bquarks_from_higgs [[17, 18, 19, 20], [16, 17, 18, 19], [17, ... [16, 17, 18, 19], [25, 26, 27, 28]] [[5, -5, 5, -5], [5, -5, 5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]]


In [35]:
# check that bquarks_from_higgs are -5 and 5
ak.any(ak.sum(higgs.children.pdgId, axis=2)!=0)


False

In [36]:
children_idxG = ak.without_parameters(genpart.childrenIdxG, behavior={})
children_idxG_flat = ak.flatten(children_idxG, axis=1)
genpart_pdgId_flat = ak.flatten(
    ak.without_parameters(genpart.pdgId, behavior={}), axis=1
)
genpart_LastCopy_flat = ak.flatten(
    ak.without_parameters(genpart.hasFlags(["isLastCopy"]), behavior={}), axis=1
)
genpart_pt_flat = ak.flatten(ak.without_parameters(genpart.pt, behavior={}), axis=1)
genparts_flat = ak.flatten(genpart)
genpart_offsets = np.concatenate(
    [[0], np.cumsum(ak.to_numpy(ak.num(genpart, axis=1), allow_missing=True))]
)
local_index_all = ak.local_index(genpart, axis=1)
local_index_b = ak.local_index(bquarks_from_higgs, axis=1)
b_quark_idx = ak.to_numpy(bquarks_from_higgs.index + genpart_offsets[:-1], allow_missing=False)
b_quarks_pdgId = ak.to_numpy(bquarks_from_higgs.pdgId, allow_missing=False)
nevents = b_quark_idx.shape[0]
firstgenpart_idxG = ak.firsts(genpart[:, 0].children).genPartIdxMotherG
firstgenpart_idxG_numpy = ak.to_numpy(firstgenpart_idxG, allow_missing=False)

In [37]:
@njit
def get_quark_last_copy(
    b_quarks_idx,
    b_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
    firstgenpart_idxG_numpy,
):
    prints = True
    # print input array
    if prints:
        print("b_quarks_idx", b_quarks_idx)
        print("b_quarks_pdgId", b_quarks_pdgId)
        print("children_idxG_flat", children_idxG_flat)
        print("genpart_pdgId_flat", genpart_pdgId_flat)
        print("genpart_offsets", genpart_offsets)
        print("genpart_LastCopy_flat", genpart_LastCopy_flat)
        print("nevents", nevents)

    # get the children ofthe b_quarks which have the same pdgId of the mother iteratively until we reach the last copy

    out = np.zeros(b_quarks_idx.shape, dtype="int64") - 1

    for iev in range(b_quarks_idx.shape[0]):

        prints=True if iev==700 else False
        if prints:
            print("\nEvent", iev)
        for ipart in range(b_quarks_idx.shape[1]):
            p_id = b_quarks_idx[iev][ipart]
            if prints:
                print("Parton", ipart, genpart_LastCopy_flat[p_id], len(children_idxG_flat[p_id]), p_id)
            i = 0
            if genpart_LastCopy_flat[p_id] or len(children_idxG_flat[p_id]) == 0:
                out[iev][ipart] = p_id
                if prints:
                    print("out", out[iev][ipart], out[iev][ipart]-genpart_offsets[iev])
                continue
            while (not genpart_LastCopy_flat[p_id]) and (not len(children_idxG_flat[p_id]) == 0) and i<5:
                i += 1
                children_idxs = reverse_index_array(
                    children_idxG_flat[p_id],
                    firstgenpart_idxG_numpy,
                    genpart_offsets,
                    nevents,
                )
                # children_idxs = children_idxG_flat[p_id]
                if prints:
                    print(children_idxs)

                # get the children with the same pdgId as the mother with highest pt
                max_pt = -1
                max_pt_idx = p_id

                # num_pdg_equal=0
                # for child_idx in children_idxs:
                #     if genpart_pdgId_flat[child_idx] == b_quarks_pdgId[iev][ipart]:
                #         num_pdg_equal+=1
                # if num_pdg_equal==1:
                #     p_id = children_idxs[0]
                #     break
                # if prints: print("\n\n###################\n\n")

                for child_idx in children_idxs:
                    if prints:
                        print("Child", child_idx)
                    if prints:
                        print(b_quarks_pdgId[iev][ipart])
                    if prints:
                        print(genpart_pdgId_flat[child_idx])
                    if genpart_pdgId_flat[child_idx] != b_quarks_pdgId[iev][ipart]:
                        continue
                    child_pt = genpart_pt_flat[child_idx]
                    if prints:
                        print(child_pt)
                    if child_pt > max_pt:
                        max_pt_idx = child_idx
                        max_pt = child_pt

                if prints:
                    print("genpart_LastCopy_flat", genpart_LastCopy_flat[max_pt_idx])
                    print("len(children_idxG_flat[p_id])", len(children_idxG_flat[p_id]))
                if genpart_LastCopy_flat[max_pt_idx] or len(children_idxG_flat[max_pt_idx]) == 0 or max_pt==-1:
                    if prints:
                        print("Found child")
                    out[iev][ipart] = max_pt_idx
                if prints:
                    print(p_id, max_pt_idx)
                p_id = max_pt_idx
                if prints:
                    print(p_id, max_pt_idx)
                # break
                # if max_pt == -1:
                #     max_pt_idx = p_id
                if out[iev][ipart] != -1:
                    if prints:
                        print("break")
                    break
            if prints:
                print("out", out[iev][ipart], out[iev][ipart]-genpart_offsets[iev])

    return out

In [38]:
b_quark_last_idx = get_quark_last_copy(
    b_quark_idx,
    b_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
    firstgenpart_idxG_numpy,
)
print(b_quark_last_idx)
print(genpart_offsets[:-1])
print(ak.any(b_quark_last_idx==-1))


b_quarks_idx [[ 17  18  19  20]
 [ 87  88  89  90]
 [154 155 156 157]
 [217 218 219 220]
 [293 294 295 296]
 [346 347 348 349]
 [409 410 411 412]
 [501 502 503 504]
 [557 558 559 560]
 [634 635 636 637]]
b_quarks_pdgId [[ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]]
children_idxG_flat [[2, 3, 4], [], [5], [6], [11, 12, 22, 23, ... [661, 662], [], [], [], [], [], []]
genpart_pdgId_flat [21, 21, 25, 25, 21, 25, 25, 25, 25, ... 11, 411, -431, 411, -421, -4122, -13, 14]
genpart_offsets [  0  71 137 202 277 327 391 483 541 609 663]
genpart_LastCopy_flat [False, False, False, False, False, False, ... True, True, True, True, True, True]
nevents 10
[[ 33  37  40  42]
 [ 98  92 103 104]
 [168 170 166 158]
 [222 218 228 232]
 [297 299 302 303]
 [356 347 354 350]
 [430 432 420 415]
 [514 505 518 521]
 [564 565 568 570]
 [642 643 640 638]]
[  0  71 137 202 277 327 391 483 541 609]
False


In [39]:
for i in range(len(b_quark_last_idx)):
    for j in range(len(b_quark_last_idx[i])):
        if b_quark_last_idx[i][j] == -1:
            print(b_quark_last_idx[i][j], b_quark_last_idx[i][j]-genpart_offsets[i], b_quarks_pdgId[i][j], genpart_pdgId_flat[b_quark_last_idx[i][j]])
            print(i, j)

In [40]:
bq=genparts_flat[b_quark_last_idx]
print(bq.index, bq.pdgId, bq.genPartIdxMother, bq.pt)
print(bquarks_last.index, bquarks_last.pdgId, bquarks_last.genPartIdxMother, bquarks_last.pt, bquarks_last.hasFlags(["fromHardProcess"]))

[[33, 37, 40, 42], [27, 21, 32, 33], [31, ... [23, 24, 27, 29], [33, 34, 31, 29]] [[5, -5, 5, -5], [5, -5, 5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] [[17, 18, 19, 20], [16, 17, 20, 19], [17, ... [16, 20, 18, 19], [25, 26, 27, 28]] [[116, 124, 186, 13.8], [35.4, 87.2, 103, ... 75.5, 40.6], [49.4, 60.5, 132, 27.6]]
[[33, 37, 40, 42], [21, 27, 32, 33], [21, ... [23, 24, 27, 29], [29, 31, 33, 34]] [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [-5, 5, 5, -5]] [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [16, 20, 18, 19], [28, 27, 25, 26]] [[116, 124, 186, 13.8], [87.2, 35.4, 103, ... 75.5, 40.6], [27.6, 132, 49.4, 60.5]] [[True, True, True, True], [True, True, ... True, True], [True, True, True, True]]


In [41]:
print("same index", ak.all(ak.sum(bq.index, axis=1)==ak.sum(bquarks_last.index, axis=1)))
bq_sum=ak.sum(bq.index, axis=1)
bquarks_last_sum=ak.sum(bquarks_last.index, axis=1)
num_gen_part=ak.num(genpart, axis=1)

same index False


In [42]:
for i in range (len(bq_sum)):
    print(bq_sum[i], bquarks_last_sum[i],bq.index[i], bquarks_last.index[i], num_gen_part[i],genpart_offsets[i])

152 152 [33, 37, 40, 42] [33, 37, 40, 42] 71 0
113 113 [27, 21, 32, 33] [21, 27, 32, 33] 66 71
114 114 [31, 33, 29, 21] [21, 29, 31, 33] 65 137
92 99 [20, 16, 26, 30] [20, 23, 26, 30] 75 202
93 93 [20, 22, 25, 26] [20, 22, 25, 26] 50 277
99 109 [29, 20, 27, 23] [23, 27, 29, 30] 64 327
133 133 [39, 41, 29, 24] [24, 29, 39, 41] 92 391
126 126 [31, 22, 35, 38] [22, 31, 35, 38] 58 483
103 103 [23, 24, 27, 29] [23, 24, 27, 29] 68 541
127 127 [33, 34, 31, 29] [29, 31, 33, 34] 54 609


In [43]:
genp=genparts_flat[218]
print(genp.index, genp.pdgId, genp.genPartIdxMother, genp.pt, genp.childrenIdxG, genp.children, genp.hasFlags(["isLastCopy"]), genp.hasFlags(["fromHardProcess"]))

16 -5 13 149.0 [] [] False True


In [44]:
gen_old=genparts_flat[225]
print(gen_old.index, gen_old.pdgId, gen_old.genPartIdxMother, gen_old.pt, gen_old.childrenIdxG, gen_old.children, gen_old.hasFlags(["isLastCopy"]), gen_old.hasFlags(["fromHardProcess"]))

23 -5 19 148.5 [] [] True True


In [45]:
# higgs_children=genparts_flat[higgs.childrenIdxG]
print(higgs.childrenIdxG)
print(ak.flatten(higgs.childrenIdxG))
higgs_children_flat2=ak.flatten(higgs.childrenIdxG, axis=None)
print(higgs_children_flat2)
print(genparts_flat)
higgs_children=genparts_flat[higgs_children_flat2]
print(higgs_children.index, higgs_children.pdgId, higgs_children.genPartIdxMother, higgs_children.pt, higgs_children.childrenIdxG, higgs_children.children, higgs_children.hasFlags(["isLastCopy"]), higgs_children.hasFlags(["fromHardProcess"]))

[[[17, 18], [19, 20]], [[89, 90], [87, ... [559, 560]], [[636, 637], [634, 635]]]
[[17, 18], [19, 20], [89, 90], [87, 88, ... 558], [559, 560], [636, 637], [634, 635]]
[17, 18, 19, 20, 89, 90, 87, 88, 154, ... 557, 558, 559, 560, 636, 637, 634, 635]
[GenParticle, GenParticle, GenParticle, ... GenParticle, GenParticle, GenParticle]
[17, 18, 19, 20, 18, 19, 16, 17, 17, 18, ... 20, 21, 16, 17, 18, 19, 27, 28, 25, 26] [5, -5, 5, -5, 5, -5, 5, -5, 5, -5, 5, -5, ... -5, 5, -5, 5, -5, 5, -5, 5, -5, 5, -5] [15, 15, 16, 16, 15, 15, 14, 14, 15, 15, ... 17, 17, 14, 14, 15, 15, 24, 24, 23, 23] [128, 164, 218, 21.7, 105, 31.5, 44, 93, ... 123, 92.8, 99.8, 144, 37.1, 55.2, 57.6] [[33, 34, 35, 36], [37], [40, 41, 43, 44, ... [638, 639], [642], [643, 644, 645]] [[GenParticle, GenParticle, GenParticle, ... GenParticle, GenParticle, GenParticle]] [False, False, False, False, False, False, ... False, False, False, False, False] [True, True, True, True, True, True, True, ... True, True, True, True, True, 

In [46]:
print(b_quarks.index, b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(bquarks_from_higgs.index, bquarks_from_higgs.pdgId, bquarks_from_higgs.genPartIdxMother, bquarks_from_higgs.pt)


[[17, 18, 19, 20], [17, 16, 18, 19], [20, ... [16, 17, 18, 19], [28, 27, 25, 26]] [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [-5, 5, 5, -5]] [[15, 15, 16, 16], [14, 14, 15, 15], [16, ... [14, 14, 15, 15], [24, 24, 23, 23]] [[128, 164, 218, 21.7], [93, 44, 105, ... 123, 92.8, 99.8], [37.1, 144, 55.2, 57.6]]
[[17, 18, 19, 20], [16, 17, 18, 19], [17, ... [16, 17, 18, 19], [25, 26, 27, 28]] [[5, -5, 5, -5], [5, -5, 5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] [[15, 15, 16, 16], [14, 14, 15, 15], [15, ... [14, 14, 15, 15], [23, 23, 24, 24]] [[128, 164, 218, 21.7], [44, 93, 105, ... 123, 92.8, 99.8], [55.2, 57.6, 144, 37.1]]


In [47]:
bquarks_from_higgs.statusFlags

<Array [[22913, 4481, 22913, ... 4481, 22913]] type='10 * var * uint16[parameter...'>

In [48]:
print(genpart.childrenIdxG)
print(~genpart.hasFlags(["isLastCopy"]), genpart.hasFlags(["isLastCopy"]))
condition=(~genpart.hasFlags(["isLastCopy"])) & ak.num(genpart.childrenIdxG, axis=2)==0
print(condition)
genpart_condition=genpart[condition]
print(genpart_condition.index, genpart_condition.pdgId, genpart_condition.genPartIdxMother, genpart_condition.pt, genpart_condition.childrenIdxG, genpart_condition.children, genpart_condition.hasFlags(["isLastCopy"]), genpart_condition.hasFlags(["fromHardProcess"]))

[[[2, 3, 4], [], [5], [6], [11, 12, 22, ... [661, 662], [], [], [], [], [], []]]
[[True, True, True, True, True, True, ... False, False, False, False, False, False]] [[False, False, False, False, False, False, ... True, True, True, True, True, True]]
[[False, True, False, False, False, False, ... True, True, True, True, True, True]]
[[1, 11, 12, 15, 16, 17, 19, 21, 22, 23, ... 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]] [[21, 21, 21, 25, 25, 5, 5, 3, 21, ... 11, 411, -431, 411, -421, -4122, -13, 14]] [[-1, 4, 4, 13, 14, 15, 16, 11, 4, 4, 4, ... 39, 40, 40, 40, 41, 41, 43, 44, 47, 47]] [[0, 24.4, 14.6, 265, 209, 128, 218, ... 49.8, 32.8, 14.9, 25.2, 11.6, 7.58, 10.9]] [[[], [21], [29], [17, 18], [19, 20], ... [661, 662], [], [], [], [], [], []]] [[[], [GenParticle], [GenParticle], [GenParticle, ... [], [], [], [], []]] [[False, True, True, True, True, False, ... True, True, True, True, True, True]] [[True, True, True, True, True, True, ... False, False, False, False, False, False]]


In [49]:
#match bq and Jet

matched_bquarks_higgs, matched_jets_higgs, deltaR_matched_higgs = (
    object_matching(
        bq,
        events.Jet,
        dr_min=0.4,
    )
)
print(bq.pt, bq.eta, bq.phi, bq.mass)
print(events.Jet.pt, events.Jet.eta, events.Jet.phi, events.Jet.mass)
print(matched_bquarks_higgs)
print(matched_jets_higgs)

for i in range(len(matched_bquarks_higgs)):
    print("\n quark", bq[i].pt, bq[i].eta, bq[i].phi, bq[i].mass)
    print("matched quark", matched_bquarks_higgs[i].pt, matched_bquarks_higgs[i].eta, matched_bquarks_higgs[i].phi, matched_bquarks_higgs[i].mass)
    print("jet", events.Jet[i].pt, events.Jet[i].eta, events.Jet[i].phi, events.Jet[i].mass)
    print("matched jet", matched_jets_higgs[i].pt, matched_jets_higgs[i].eta, matched_jets_higgs[i].phi, matched_jets_higgs[i].mass)


[[116, 124, 186, 13.8], [35.4, 87.2, 103, ... 75.5, 40.6], [49.4, 60.5, 132, 27.6]] [[0.963, 0.908, 0.928, 1.57], [-0.65, -1.61, ... -0.721], [-0.468, 1.21, 1.74, 2.09]] [[-0.98, -1.84, 1.77, -2.54], [-2.76, -0.873, ... -2.36], [2.1, 1.08, -2.36, -0.451]] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, ... 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[201, 141, 140, 26.8, 17, 16.7, 15.6], ... [100, 58.2, 36.1, 30, 24.3, 23.2]] [[0.941, 0.891, 0.944, 0.44, -1.06, -2.76, ... -0.472, -0.0853, 2.08, -1.41, 1.21]] [[1.76, -1.83, -0.973, -1.57, 0.947, 1.68, ... 2.08, -0.37, -0.344, -0.746, 1.11]] [[16, 17.5, 15.3, 4.78, 3.88, 2.18, 3.5], ... [8.21, 6.76, 6.37, 5.69, 4.6, 3.75]]
[[GenParticle, GenParticle, GenParticle, None, ... GenParticle, None, GenParticle]]
[[Jet, Jet, Jet, None, None, None, None], ... [Jet, Jet, None, Jet, None, Jet]]

 quark [116, 124, 186, 13.8] [0.963, 0.908, 0.928, 1.57] [-0.98, -1.84, 1.77, -2.54] [0, 0, 0, 0]
matched quark [186, 124, 116, None, None, None, None] [0.928, 0.908,

In [50]:
#match bquarks_last and Jet

matched_bquarks_lastuarks_higgs, matched_jets_higgs, deltaR_matched_higgs = (
    object_matching(
        bquarks_last,
        events.Jet,
        dr_min=0.4,
    )
)
print(bquarks_last.pt, bquarks_last.eta, bquarks_last.phi, bquarks_last.mass)
print(events.Jet.pt, events.Jet.eta, events.Jet.phi, events.Jet.mass)
print(matched_bquarks_lastuarks_higgs)
print(matched_jets_higgs)

for i in range(len(matched_bquarks_lastuarks_higgs)):
    print("\n quark", bquarks_last[i].pt, bquarks_last[i].eta, bquarks_last[i].phi, bquarks_last[i].mass)
    print("matched quark", matched_bquarks_lastuarks_higgs[i].pt, matched_bquarks_lastuarks_higgs[i].eta, matched_bquarks_lastuarks_higgs[i].phi, matched_bquarks_lastuarks_higgs[i].mass)
    print("jet", events.Jet[i].pt, events.Jet[i].eta, events.Jet[i].phi, events.Jet[i].mass)
    print("matched jet", matched_jets_higgs[i].pt, matched_jets_higgs[i].eta, matched_jets_higgs[i].phi, matched_jets_higgs[i].mass)


[[116, 124, 186, 13.8], [87.2, 35.4, 103, ... 75.5, 40.6], [27.6, 132, 49.4, 60.5]] [[0.963, 0.908, 0.928, 1.57], [-1.61, -0.65, ... -0.721], [2.09, 1.74, -0.468, 1.21]] [[-0.98, -1.84, 1.77, -2.54], [-0.873, -2.76, ... -2.36], [-0.451, -2.36, 2.1, 1.08]] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, ... 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[201, 141, 140, 26.8, 17, 16.7, 15.6], ... [100, 58.2, 36.1, 30, 24.3, 23.2]] [[0.941, 0.891, 0.944, 0.44, -1.06, -2.76, ... -0.472, -0.0853, 2.08, -1.41, 1.21]] [[1.76, -1.83, -0.973, -1.57, 0.947, 1.68, ... 2.08, -0.37, -0.344, -0.746, 1.11]] [[16, 17.5, 15.3, 4.78, 3.88, 2.18, 3.5], ... [8.21, 6.76, 6.37, 5.69, 4.6, 3.75]]
[[GenParticle, GenParticle, GenParticle, None, ... GenParticle, None, GenParticle]]
[[Jet, Jet, Jet, None, None, None, None], ... [Jet, Jet, None, Jet, None, Jet]]

 quark [116, 124, 186, 13.8] [0.963, 0.908, 0.928, 1.57] [-0.98, -1.84, 1.77, -2.54] [0, 0, 0, 0]
matched quark [186, 124, 116, None, None, None, None] [0.928, 0.908,

In [9]:
print(events.GenPart.index[0])
events.genpart_mask=events.GenPart.hasFlags(["isLastCopy"])
print(events.genpart_mask)
events.genpart_masked=events.GenPart[events.genpart_mask]
print(events.genpart_masked.index[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[[False, False, False, False, False, False, ... True, True, True, True, True, True]]
[11, 12, 15, 16, 21, 22, 23, 24, 25, 26, ... 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
